In [1]:
import json
import os
import time
import random
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torchvision import models
from torchsummary import summary
from sklearn.model_selection import train_test_split

from model_pytorch import EfficientNet
from utils import Bar,Logger, AverageMeter, accuracy, mkdir_p, savefig
from warmup_scheduler import GradualWarmupScheduler

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import collections

In [2]:
# GPU Device
gpu_id = 0
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
use_cuda = torch.cuda.is_available()
print("GPU device %d:" %(gpu_id), use_cuda)

GPU device 0: True


# Arguments

In [3]:
data_dir = '/media/data2/dataset/GAN_ImageData/PGGAN_128/'

In [4]:
pretrained = ''
resume = './log/pggan/64/b1/checkpoint.pth.tar'

In [5]:
# Model
model_name = 'efficientnet-b1' # b0-b7 scale

# Optimization
num_classes = 2
epochs = 300
start_epoch = 0
train_batch = 756
test_batch = 500
lr = 0.1
schedule = [20, 125, 225, 275]
momentum = 0.9
gamma = 0.1 # LR is multiplied by gamma on schedule

# CheckPoint
checkpoint = './log/pggan/64/b1' # dir
if not os.path.isdir(checkpoint):
    os.mkdir(checkpoint)
num_workers = 4

# Seed
manual_seed = 7
random.seed(manual_seed)
torch.cuda.manual_seed_all(manual_seed)

# Image
size = (64, 64)

best_acc = 0

In [6]:
state = {}
state['num_classes'] = num_classes
state['epochs'] = epochs
state['start_epoch'] = start_epoch
state['train_batch'] = train_batch
state['test_batch'] = test_batch
state['lr'] = lr
state['schedule'] = schedule
state['momentum'] = momentum
state['gamma'] = gamma

# Dataset

In [7]:
train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'validation')    
train_aug = transforms.Compose([
#     transforms.RandomAffine(degrees=2, translate=(0.02, 0.02), scale=(0.98, 1.02), shear=2, fillcolor=(124,117,104)),
    transforms.Resize(size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.RandomErasing(p=0.3, scale=(0.02, 0.10), ratio=(0.3, 3.3), value=0, inplace=True),
])
val_aug = transforms.Compose([
    transforms.Resize(size),
    transforms.ToTensor(),
])

# pin_memory : cuda pin memeory use
train_loader = DataLoader(datasets.ImageFolder(train_dir, transform=train_aug),
                          batch_size=train_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_loader = DataLoader(datasets.ImageFolder(val_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)

# Model

In [8]:
model = EfficientNet.from_name(model_name, num_classes=num_classes)

# Pre-trained
if pretrained:
    print("=> using pre-trained model '{}'".format(pretrained))
    model.load_state_dict(torch.load(pretrained)['state_dict'])

In [9]:
model.to('cuda')
cudnn.benchmark = True
print('    Total params: %.2fM' % (sum(p.numel() for p in model.parameters())/1000000.0))

    Total params: 6.52M


In [10]:
# summary(model, input_size=(3,64,64), device='cuda')

# Loss

In [11]:
criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=1e-4, nesterov=True)
# optimizer = optim.Adam(model.parameters(), weight_decay=1e-4)
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
scheduler_warmup = GradualWarmupScheduler(optimizer, multiplier=8, total_epoch=10, after_scheduler=scheduler_cosine)

In [12]:
# Resume
if resume:
    print('==> Resuming from checkpoint..')
    checkpoint = os.path.dirname(resume)
#     checkpoint = torch.load(resume)
    resume = torch.load(resume)
    best_acc = resume['best_acc']
    start_epoch = resume['epoch']
    model.load_state_dict(resume['state_dict'])
    optimizer.load_state_dict(resume['optimizer'])
    logger = Logger(os.path.join(checkpoint, 'log.txt'), resume=True)
else:
    logger = Logger(os.path.join(checkpoint, 'log.txt'))
    logger.set_names(['Learning Rate', 'Train Loss', 'Valid Loss', 'Train Acc.', 'Valid Acc.'])

==> Resuming from checkpoint..


# Train

In [13]:
def train(train_loader, model, criterion, optimizer, epoch, use_cuda):
    model.train()
    torch.set_grad_enabled(True)
    
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    end = time.time()
    
    bar = Bar('Processing', max=len(train_loader))
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        batch_size = inputs.size(0)
        if batch_size < train_batch:
            continue
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)

        # compute output
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:.4f} | top1: {top1: .4f} '.format(
                    batch=batch_idx + 1,
                    size=len(train_loader),
                    data=data_time.val,
                    bt=batch_time.val,
                    total=bar.elapsed_td,
                    eta=bar.eta_td,
                    loss=losses.avg,
                    top1=top1.avg,
                    )
        bar.next()
        if batch_idx % 10 == 0:
            print('{batch}/{size} Data:{data:.3f} | Batch:{bt:.3f} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
                 batch=batch_idx+1, size=len(train_loader), data=data_time.val, bt=batch_time.val, total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    bar.finish()
    return (losses.avg, top1.avg)

In [14]:
def test(val_loader, model, criterion, epoch, use_cuda):
    global best_acc

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()
    torch.set_grad_enabled(False)

    end = time.time()
    bar = Bar('Processing', max=len(val_loader))
    for batch_idx, (inputs, targets) in enumerate(val_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs, volatile=True), torch.autograd.Variable(targets)

        # compute output
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:} | top1: {top1:}'.format(
                    batch=batch_idx + 1,
                    size=len(val_loader),
                    data=data_time.avg,
                    bt=batch_time.avg,
                    total=bar.elapsed_td,
                    eta=bar.eta_td,
                    loss=losses.avg,
                    top1=top1.avg,)
        bar.next()
    print('{batch}/{size} Data:{data:.3f} | Batch:{bt:.3f} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
         batch=batch_idx+1, size=len(val_loader), data=data_time.val, bt=batch_time.val, total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    bar.finish()
    return (losses.avg, top1.avg)

In [15]:
def save_checkpoint(state, is_best, checkpoint='checkpoint', filename='checkpoint.pth.tar'):
    filepath = os.path.join(checkpoint, filename)
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'model_best.pth.tar'))

def adjust_learning_rate(optimizer, epoch):
    global state
    lr_set = [1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
    lr_list = schedule.copy()
    lr_list.append(epoch)
    lr_list.sort()
    idx = lr_list.index(epoch)
    state['lr'] *= lr_set[idx]
    for param_group in optimizer.param_groups:
        param_group['lr'] = state['lr']

In [ ]:
for epoch in range(start_epoch, epochs):
    state['lr'] = optimizer.state_dict()['param_groups'][0]['lr']
    adjust_learning_rate(optimizer, epoch)
    print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))
    train_loss, train_acc = train(train_loader, model, criterion, optimizer, epoch, use_cuda)
    test_loss, test_acc = test(val_loader, model, criterion, epoch, use_cuda)
    
    logger.append([state['lr'], train_loss, test_loss, train_acc, test_acc])
    scheduler_warmup.step()

    is_best = test_acc > best_acc
    best_acc = max(test_acc, best_acc)
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict' : model.state_dict(),
        'acc': test_acc,
        'best_acc': best_acc,
        'optimizer': optimizer.state_dict(),
    }, is_best, checkpoint=checkpoint)


Epoch: [20 | 300] LR: 0.798597
1/170 Data:4.332 | Batch:10.476 | Total:0:00:10 | ETA:0:29:33 | Loss:0.6902084946632385 | top1:50.925926208496094
11/170 Data:0.004 | Batch:0.918 | Total:0:00:20 | ETA:0:05:19 | Loss:0.6760055747899142 | top1:59.018760681152344
21/170 Data:0.002 | Batch:1.041 | Total:0:00:31 | ETA:0:02:36 | Loss:0.6849195134072077 | top1:57.5585823059082
31/170 Data:0.014 | Batch:0.939 | Total:0:00:41 | ETA:0:02:20 | Loss:0.67562355918269 | top1:59.464073181152344
41/170 Data:0.001 | Batch:1.041 | Total:0:00:51 | ETA:0:02:11 | Loss:0.6609067611578034 | top1:61.543426513671875
51/170 Data:0.012 | Batch:1.033 | Total:0:01:01 | ETA:0:01:57 | Loss:0.6523556101555917 | top1:62.37939453125
61/170 Data:0.006 | Batch:0.921 | Total:0:01:10 | ETA:0:01:48 | Loss:0.6416993551566953 | top1:63.60048294067383
71/170 Data:0.003 | Batch:1.031 | Total:0:01:20 | ETA:0:01:39 | Loss:0.6332411337906206 | top1:64.45152282714844
81/170 Data:0.025 | Batch:1.005 | Total:0:01:30 | ETA:0:01:29 | Lo

/home/cutz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


65/65 Data:0.002 | Batch:0.646 | Total:0:00:30 | ETA:0:00:00 | Loss:0.3021375691036569 | top1:91.76324462890625

Epoch: [21 | 300] LR: 0.170000
1/170 Data:4.332 | Batch:6.021 | Total:0:00:06 | ETA:0:17:01 | Loss:0.498027503490448 | top1:76.984130859375
11/170 Data:0.002 | Batch:0.948 | Total:0:00:15 | ETA:0:03:56 | Loss:0.5120180926539681 | top1:74.98797607421875
21/170 Data:0.001 | Batch:0.981 | Total:0:00:23 | ETA:0:01:55 | Loss:0.5128738624708993 | top1:74.68506622314453
31/170 Data:0.001 | Batch:0.672 | Total:0:00:30 | ETA:0:01:42 | Loss:0.5097551393893457 | top1:74.82505798339844
41/170 Data:0.011 | Batch:0.846 | Total:0:00:37 | ETA:0:01:28 | Loss:0.5110867372373256 | top1:74.79352569580078
51/170 Data:0.001 | Batch:0.877 | Total:0:00:45 | ETA:0:01:30 | Loss:0.5095378078666388 | top1:74.86772155761719
61/170 Data:0.011 | Batch:0.539 | Total:0:00:51 | ETA:0:01:18 | Loss:0.5089464959551077 | top1:74.86988830566406
71/170 Data:0.001 | Batch:0.926 | Total:0:00:58 | ETA:0:01:06 | Loss:

65/65 Data:0.000 | Batch:0.134 | Total:0:00:31 | ETA:0:00:00 | Loss:0.272040157674629 | top1:94.00312042236328

Epoch: [25 | 300] LR: 0.045000
1/170 Data:3.999 | Batch:5.098 | Total:0:00:05 | ETA:0:14:22 | Loss:0.45640623569488525 | top1:79.36507415771484
11/170 Data:0.013 | Batch:1.085 | Total:0:00:15 | ETA:0:02:43 | Loss:0.48982104659080505 | top1:76.3468017578125
21/170 Data:0.025 | Batch:0.834 | Total:0:00:24 | ETA:0:02:15 | Loss:0.4868604711123875 | top1:76.51802062988281
31/170 Data:0.001 | Batch:0.576 | Total:0:00:31 | ETA:0:01:51 | Loss:0.4860553943341778 | top1:76.5873031616211
41/170 Data:0.001 | Batch:0.687 | Total:0:00:39 | ETA:0:01:35 | Loss:0.4846529233746412 | top1:76.57440185546875
51/170 Data:0.001 | Batch:0.799 | Total:0:00:47 | ETA:0:01:38 | Loss:0.4827803589549719 | top1:76.7532958984375
61/170 Data:0.001 | Batch:0.862 | Total:0:00:55 | ETA:0:01:26 | Loss:0.48353952169418335 | top1:76.73908996582031
71/170 Data:0.008 | Batch:0.757 | Total:0:01:03 | ETA:0:01:23 | Los

65/65 Data:0.002 | Batch:0.068 | Total:0:00:22 | ETA:0:00:00 | Loss:0.2419567804934451 | top1:95.86605072021484

Epoch: [29 | 300] LR: 0.073000
1/170 Data:2.209 | Batch:3.151 | Total:0:00:03 | ETA:0:08:53 | Loss:0.4785482883453369 | top1:76.984130859375
11/170 Data:0.001 | Batch:1.029 | Total:0:00:13 | ETA:0:02:43 | Loss:0.46690620346502826 | top1:77.77777862548828
21/170 Data:0.006 | Batch:1.068 | Total:0:00:23 | ETA:0:02:34 | Loss:0.4649209365958259 | top1:77.7273941040039
31/170 Data:0.001 | Batch:0.988 | Total:0:00:34 | ETA:0:02:26 | Loss:0.46425510125775493 | top1:77.76497650146484
41/170 Data:0.008 | Batch:0.990 | Total:0:00:44 | ETA:0:02:12 | Loss:0.4623669385910034 | top1:77.92295837402344
51/170 Data:0.001 | Batch:1.040 | Total:0:00:54 | ETA:0:02:03 | Loss:0.4623441479954065 | top1:77.88671112060547
61/170 Data:0.001 | Batch:1.029 | Total:0:01:05 | ETA:0:01:54 | Loss:0.46198576249060086 | top1:77.86668395996094
71/170 Data:0.003 | Batch:0.963 | Total:0:01:15 | ETA:0:01:42 | Lo

...

65/65 Data:0.000 | Batch:0.103 | Total:0:00:22 | ETA:0:00:00 | Loss:0.1928833807554572 | top1:97.71028137207031

Epoch: [41 | 300] LR: 0.079781
1/170 Data:3.142 | Batch:3.741 | Total:0:00:03 | ETA:0:10:33 | Loss:0.41829296946525574 | top1:79.36507415771484
11/170 Data:0.002 | Batch:1.007 | Total:0:00:12 | ETA:0:02:21 | Loss:0.4339489611712369 | top1:79.42520904541016
21/170 Data:0.010 | Batch:1.016 | Total:0:00:22 | ETA:0:02:30 | Loss:0.441328444651195 | top1:78.93046569824219
31/170 Data:0.003 | Batch:1.064 | Total:0:00:32 | ETA:0:02:19 | Loss:0.43960637334854374 | top1:79.10479736328125
41/170 Data:0.001 | Batch:1.042 | Total:0:00:42 | ETA:0:02:13 | Loss:0.4413329442826713 | top1:79.0360107421875
51/170 Data:0.019 | Batch:0.957 | Total:0:00:53 | ETA:0:02:03 | Loss:0.4415793547443315 | top1:79.0616226196289
61/170 Data:0.004 | Batch:1.015 | Total:0:01:03 | ETA:0:01:52 | Loss:0.44039114723440076 | top1:79.14173126220703
71/170 Data:0.010 | Batch:1.048 | Total:0:01:13 | ETA:0:01:41 | Lo

65/65 Data:0.000 | Batch:0.131 | Total:0:00:22 | ETA:0:00:00 | Loss:0.1945476216319194 | top1:97.99688720703125

Epoch: [45 | 300] LR: 0.079571
1/170 Data:2.625 | Batch:3.664 | Total:0:00:03 | ETA:0:10:20 | Loss:0.45278170704841614 | top1:77.51322174072266
11/170 Data:0.001 | Batch:1.046 | Total:0:00:14 | ETA:0:02:45 | Loss:0.4358506283976815 | top1:79.50938415527344
21/170 Data:0.001 | Batch:0.972 | Total:0:00:24 | ETA:0:02:34 | Loss:0.4326822119099753 | top1:79.65483093261719
31/170 Data:0.001 | Batch:1.025 | Total:0:00:34 | ETA:0:02:24 | Loss:0.43555475050403225 | top1:79.34801483154297
41/170 Data:0.001 | Batch:0.968 | Total:0:00:44 | ETA:0:02:08 | Loss:0.43557372253115584 | top1:79.25862121582031
51/170 Data:0.002 | Batch:1.031 | Total:0:00:54 | ETA:0:02:02 | Loss:0.4358457636599447 | top1:79.2820816040039
61/170 Data:0.001 | Batch:0.975 | Total:0:01:04 | ETA:0:01:53 | Loss:0.43451755408380854 | top1:79.36724853515625
71/170 Data:0.001 | Batch:1.005 | Total:0:01:15 | ETA:0:01:43 |

65/65 Data:0.002 | Batch:0.073 | Total:0:00:20 | ETA:0:00:00 | Loss:0.17558436903441063 | top1:98.14642333984375

Epoch: [49 | 300] LR: 0.079291
1/170 Data:1.850 | Batch:2.894 | Total:0:00:02 | ETA:0:08:10 | Loss:0.43624868988990784 | top1:79.36507415771484
11/170 Data:0.014 | Batch:1.035 | Total:0:00:13 | ETA:0:02:45 | Loss:0.4281705319881439 | top1:79.79798126220703
21/170 Data:0.002 | Batch:1.083 | Total:0:00:23 | ETA:0:02:38 | Loss:0.4253745249339512 | top1:79.95087432861328
31/170 Data:0.020 | Batch:1.004 | Total:0:00:33 | ETA:0:02:22 | Loss:0.4248467193495843 | top1:79.92831420898438
41/170 Data:0.007 | Batch:1.127 | Total:0:00:44 | ETA:0:02:17 | Loss:0.4235903715215078 | top1:80.03614044189453
51/170 Data:0.002 | Batch:0.940 | Total:0:00:54 | ETA:0:02:03 | Loss:0.4238978787964466 | top1:79.94605255126953
61/170 Data:0.001 | Batch:0.977 | Total:0:01:04 | ETA:0:01:53 | Loss:0.42533107077489135 | top1:79.83779907226562
71/170 Data:0.002 | Batch:1.009 | Total:0:01:15 | ETA:0:01:43 |

65/65 Data:0.000 | Batch:0.068 | Total:0:00:20 | ETA:0:00:00 | Loss:0.18836487927169443 | top1:98.45171356201172

Epoch: [53 | 300] LR: 0.078943
1/170 Data:2.782 | Batch:3.319 | Total:0:00:03 | ETA:0:09:21 | Loss:0.3974355459213257 | top1:82.53968048095703
11/170 Data:0.001 | Batch:0.520 | Total:0:00:09 | ETA:0:02:29 | Loss:0.42060663212429394 | top1:79.9422836303711
21/170 Data:0.024 | Batch:1.042 | Total:0:00:18 | ETA:0:02:08 | Loss:0.4214260535580771 | top1:80.05795288085938
31/170 Data:0.003 | Batch:1.034 | Total:0:00:28 | ETA:0:02:22 | Loss:0.4218938417973057 | top1:79.84297943115234
41/170 Data:0.002 | Batch:1.018 | Total:0:00:38 | ETA:0:02:14 | Loss:0.4209526973526652 | top1:80.01355743408203
51/170 Data:0.012 | Batch:1.016 | Total:0:00:49 | ETA:0:02:02 | Loss:0.42102773341478084 | top1:80.08869934082031
61/170 Data:0.038 | Batch:1.016 | Total:0:00:59 | ETA:0:01:53 | Loss:0.4225611657392783 | top1:79.99392700195312
71/170 Data:0.002 | Batch:1.072 | Total:0:01:09 | ETA:0:01:44 | 

161/170 Data:0.002 | Batch:0.955 | Total:0:02:41 | ETA:0:00:10 | Loss:0.4217404497336157 | top1:79.94922637939453
65/65 Data:0.000 | Batch:0.063 | Total:0:00:20 | ETA:0:00:00 | Loss:0.16125894187024079 | top1:98.47663879394531

Epoch: [57 | 300] LR: 0.078527
1/170 Data:3.082 | Batch:4.129 | Total:0:00:04 | ETA:0:11:38 | Loss:0.44251152873039246 | top1:79.62962341308594
11/170 Data:0.001 | Batch:1.027 | Total:0:00:14 | ETA:0:02:44 | Loss:0.4241302121769298 | top1:79.84608459472656
21/170 Data:0.002 | Batch:1.072 | Total:0:00:24 | ETA:0:02:36 | Loss:0.42307927636873155 | top1:79.74301147460938
31/170 Data:0.005 | Batch:0.861 | Total:0:00:34 | ETA:0:02:24 | Loss:0.4242791216219625 | top1:79.531494140625
41/170 Data:0.002 | Batch:0.971 | Total:0:00:45 | ETA:0:02:11 | Loss:0.4258199755738421 | top1:79.46832275390625
51/170 Data:0.001 | Batch:1.063 | Total:0:00:55 | ETA:0:02:04 | Loss:0.4225845950491288 | top1:79.69706726074219
61/170 Data:0.011 | Batch:0.987 | Total:0:01:05 | ETA:0:01:50 | 